### svm 하이퍼 파라미터
* C(cost)
    - cost값이 작으면 과소적합 발생
        * cost값을 작게하면 훈련 데이터에 대해 어느정도 오류를 허용하게 되며, 새로 들어오는 데이터는 잘 분류할 수 있음
    - cost값이 커지면 과대적합 발생
        * cost값을 크게하면 훈련 데이터에 대해 오류가 최소화 되지만, 새로 들어오는 데이터는 분류를 잘못할 가능성이 높음

![img](images/cost.png)

* gamma
    - 결정 경계에 영향을 끼치는 범위를 조절하는 변수
    - gamma가 크면 과대적합이 될 수 있음
        - gamma가 크면 결정경계에 영향력이 커 결정경계가 복잡해짐
    - gamma가 작으면 과소적합이 될 수 있음
        - gamma가 작으면 결정경계에 영향력이 미미하여 결정경계가 부드러워짐

![img](images/gamma.png)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import pandas as pd

df = pd.read_csv("data/titanic_cleaning.csv")
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,0,22.0,1,0,7.2500
1,2,1,1,1,38.0,1,0,71.2833
2,3,1,3,1,26.0,0,0,7.9250
3,4,1,1,1,35.0,1,0,53.1000
4,5,0,3,0,35.0,0,0,8.0500


In [4]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
label = 'Survived'
X, y = df[features], df[label]

In [5]:
scaler = StandardScaler()
scaler.fit(X)
X_scaler = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, test_size=0.2)

In [6]:
svc = SVC(kernel="linear")
svc.fit( X_train , y_train)
print("train : ", svc.score(X_train, y_train ) )
print("test : ", svc.score(X_test, y_test ) )

train :  0.7851123595505618
test :  0.7932960893854749


### 하이퍼 파라미터

In [7]:
svc = SVC(kernel="linear", C=100 )
svc.fit( X_train, y_train )

print("train : ", svc.score(X_train, y_train ) )
print("test : ", svc.score(X_test, y_test ) )

train :  0.7851123595505618
test :  0.7932960893854749


In [8]:
param_cost = [ 0.001, 0.01, 0.1,  1.0,  10.0, 100.0 ]
train_list = []
test_list = []
cost_list = []

for cost in param_cost:
    svc = SVC(kernel="linear", C=cost)
    svc.fit( X_train, y_train)
    
    cost_list.append( cost )
    test_list.append( svc.score(X_test, y_test))
    train_list.append(svc.score(X_train, y_train))
    
dic = {"cost":cost_list, "train" :train_list, "test" : test_list}
score_df = pd.DataFrame(dic)

In [9]:
score_df

,cost,train,test
0,0.001,0.636236,0.614525
1,0.010,0.785112,0.793296
2,0.100,0.785112,0.793296
3,1.000,0.785112,0.793296
4,10.000,0.785112,0.793296
5,100.000,0.785112,0.793296


### gridSearchCV

In [11]:
from sklearn.model_selection import GridSearchCV

param_c = [ 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
param_gamma = [ 0.001, 0.01, 0.1, 1.0, 10.0 ]

params= {"C": param_c, "gamma": param_gamma}
svc = SVC(kernel="rbf")

grid_cv = GridSearchCV(svc, param_grid=params, cv=3, n_jobs=-1)
grid_cv.fit( X_train, y_train)

print("최적의 파라미터 : ", grid_cv.best_params_)

print("train : ",grid_cv.score(X_train, y_train))
print("test : ",grid_cv.score(X_test, y_test))

최적의 파라미터 :  {'C': 100.0, 'gamma': 0.01}
train :  0.827247191011236
test :  0.8324022346368715
